In [121]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


response = requests.get('https://jeffhuang.com/best_paper_awards/#2023')

if response.status_code != 200 :
    raise ValueError('Request Failed!')
            
html = response.text
soup = BeautifulSoup(html, 'html.parser')

data = []
for row in soup.find_all('tr'):
    cols = row.find_all(['th', 'td'])

    if len(cols) == 1 :
        year = cols[0].text.strip()
        continue
    elif len(cols) == 3 :
        conference = cols[0].text.strip()
        cols = cols[1:]

    paper = cols[0].text.strip()

    try : ##공동 저자가 있는 경우.
        first = [cols[1].text.split(";")[0]]
        et = re.split(r'<br/>', str((cols[1].find_all("div")[0])))
        et[0] = et[0][et[0].find(">")+1:]
        et[len(et)-1] = et[len(et)-1][:et[len(et)-1].find("<")]
        first = first + et
    except : ##공동 저자가 없는 경우.
        first = [cols[1].text.split(";")[0]]

    for authors in first :
        authors = authors.split(", ")
        if authors[0].find("&") != -1 : ##저자가 2명인 경우, et al. 이 아닌 &로 구분한 경우가 존재.
            authors[0] = authors[0].split(" & ")
            for j in authors[0] :
                append = [year, conference, paper, j, authors[1]]    
                data.append(append) 

        else :
            append = [year, conference, paper, authors[0], authors[1]]    
            data.append(append)
       

columns = ['year', 'conference', 'paper', 'author', 'institute']
df = pd.DataFrame(data, columns=columns)
df.to_csv("./data/best_paper_awards_crawl.csv", index= False)
print(df)

In [142]:
df.loc[df['author'] == 'Michael Bernstein']

,year,conference,paper,author,institute
629,2023,UIST User Interface,Generative Agents: Interactive Simulacra of Hu...,Michael Bernstein,Stanford University
716,2022,CHI Human-Computer Interaction,Jury Learning: Integrating Dissenting Voices i...,Michael Bernstein,Stanford University
2295,2019,CHI Human-Computer Interaction,Street-Level Algorithms: A Theory At The Gaps ...,Michael Bernstein,Stanford University
3076,2017,CHI Human-Computer Interaction,Flash Organizations: Crowdsourcing Complex Wor...,Michael Bernstein,Stanford University
3411,2016,CHI Human-Computer Interaction,Empath: Understanding Topic Signals in Large-S...,Michael Bernstein,Stanford University
4193,2014,UIST User Interface,Expert Crowdsourcing with Flash Teams,Michael Bernstein,Stanford University
5637,2006,UIST User Interface,Reflective Physical Prototyping through Integr...,Michael Bernstein,Stanford University
